# Imports

In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

from PIL import Image
import pydicom
import os

# Data preprocessing

In [2]:
train_df = pd.read_csv("./data/train.csv")

In [3]:
train_df

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
0,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1,4646740,Normal/Mild,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate,Normal/Mild
2,7143189,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
3,8785691,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4,10728036,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970,4282019580,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate
1971,4283570761,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1972,4284048608,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild
1973,4287160193,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Severe,Moderate,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1975 entries, 0 to 1974
Data columns (total 26 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   study_id                                1975 non-null   int64 
 1   spinal_canal_stenosis_l1_l2             1974 non-null   object
 2   spinal_canal_stenosis_l2_l3             1974 non-null   object
 3   spinal_canal_stenosis_l3_l4             1974 non-null   object
 4   spinal_canal_stenosis_l4_l5             1974 non-null   object
 5   spinal_canal_stenosis_l5_s1             1974 non-null   object
 6   left_neural_foraminal_narrowing_l1_l2   1973 non-null   object
 7   left_neural_foraminal_narrowing_l2_l3   1973 non-null   object
 8   left_neural_foraminal_narrowing_l3_l4   1973 non-null   object
 9   left_neural_foraminal_narrowing_l4_l5   1973 non-null   object
 10  left_neural_foraminal_narrowing_l5_s1   1973 non-null   object
 11  righ

In [5]:
train_label_coordinates_df = pd.read_csv("./data/train_label_coordinates.csv")

In [6]:
train_label_coordinates_df

,study_id,series_id,instance_number,condition,level,x,y
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602
...,...,...,...,...,...,...,...
48687,4290709089,4237840455,11,Left Neural Foraminal Narrowing,L1/L2,219.465940,97.831063
48688,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L2/L3,205.340599,140.207084
48689,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L3/L4,202.724796,181.013624
48690,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L4/L5,202.933333,219.733333


Let's add a column to train_label_coordinates_df with the severity that is included in the train_df

In [7]:
# Create a mapping from the level to the corresponding column name in df1
level_to_column = {
    'Spinal Canal StenosisL1/L2': 'spinal_canal_stenosis_l1_l2',
    'Spinal Canal StenosisL2/L3': 'spinal_canal_stenosis_l2_l3',
    'Spinal Canal StenosisL3/L4': 'spinal_canal_stenosis_l3_l4',
    'Spinal Canal StenosisL4/L5': 'spinal_canal_stenosis_l4_l5',
    'Spinal Canal StenosisL5/S1': 'spinal_canal_stenosis_l5_s1',
    'Left Neural Foraminal NarrowingL1/L2': 'left_neural_foraminal_narrowing_l1_l2',
    'Left Neural Foraminal NarrowingL2/L3': 'left_neural_foraminal_narrowing_l2_l3',
    'Left Neural Foraminal NarrowingL3/L4': 'left_neural_foraminal_narrowing_l3_l4',
    'Left Neural Foraminal NarrowingL4/L5': 'left_neural_foraminal_narrowing_l4_l5',
    'Left Neural Foraminal NarrowingL5/S1': 'left_neural_foraminal_narrowing_l5_s1',
    'Right Neural Foraminal NarrowingL1/L2': 'right_neural_foraminal_narrowing_l1_l2',
    'Right Neural Foraminal NarrowingL2/L3': 'right_neural_foraminal_narrowing_l2_l3',
    'Right Neural Foraminal NarrowingL3/L4': 'right_neural_foraminal_narrowing_l3_l4',
    'Right Neural Foraminal NarrowingL4/L5': 'right_neural_foraminal_narrowing_l4_l5',
    'Right Neural Foraminal NarrowingL5/S1': 'right_neural_foraminal_narrowing_l5_s1',
    'Left Subarticular StenosisL1/L2': 'left_subarticular_stenosis_l1_l2',
    'Left Subarticular StenosisL2/L3': 'left_subarticular_stenosis_l2_l3',
    'Left Subarticular StenosisL3/L4': 'left_subarticular_stenosis_l3_l4',
    'Left Subarticular StenosisL4/L5': 'left_subarticular_stenosis_l4_l5',
    'Left Subarticular StenosisL5/S1': 'left_subarticular_stenosis_l5_s1',
    'Right Subarticular StenosisL1/L2': 'right_subarticular_stenosis_l1_l2',
    'Right Subarticular StenosisL2/L3': 'right_subarticular_stenosis_l2_l3',
    'Right Subarticular StenosisL3/L4': 'right_subarticular_stenosis_l3_l4',
    'Right Subarticular StenosisL4/L5': 'right_subarticular_stenosis_l4_l5',
    'Right Subarticular StenosisL5/S1': 'right_subarticular_stenosis_l5_s1',
}

# Add a new column in train_label_coordinates_df for the severity
train_label_coordinates_df['severity'] = train_label_coordinates_df.apply(lambda row: train_df.loc[train_df['study_id'] == row['study_id'], level_to_column[row['condition'] + row['level']]].values[0], axis=1)


Let's also get the image paths into the df

In [8]:
train_label_coordinates_df["image_path"] = train_label_coordinates_df.apply(lambda row: f"./data/train_images/{row['study_id']}/{row['series_id']}/{row['instance_number']}.dcm", axis=1)

In [76]:
train_label_coordinates_df['coordinates'] = train_label_coordinates_df.apply(lambda row: (round(row['x'], 6), round(row['y'], 6)), axis=1)

In [77]:
train_label_coordinates_df

,study_id,series_id,instance_number,condition,level,x,y,severity,image_path,coordinates
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Normal/Mild,./data/train_images/4003253/702807833/8.dcm,"(322.831858, 227.964602)"
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,Normal/Mild,./data/train_images/4003253/702807833/8.dcm,"(320.571429, 295.714286)"
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,Normal/Mild,./data/train_images/4003253/702807833/8.dcm,"(323.030303, 371.818182)"
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,Normal/Mild,./data/train_images/4003253/702807833/8.dcm,"(335.292035, 427.327434)"
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,Normal/Mild,./data/train_images/4003253/702807833/8.dcm,"(353.415929, 483.964602)"
...,...,...,...,...,...,...,...,...,...,...
48687,4290709089,4237840455,11,Left Neural Foraminal Narrowing,L1/L2,219.465940,97.831063,Normal/Mild,./data/train_images/4290709089/4237840455/11.dcm,"(219.46594, 97.831063)"
48688,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L2/L3,205.340599,140.207084,Normal/Mild,./data/train_images/4290709089/4237840455/12.dcm,"(205.340599, 140.207084)"
48689,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L3/L4,202.724796,181.013624,Normal/Mild,./data/train_images/4290709089/4237840455/12.dcm,"(202.724796, 181.013624)"
48690,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L4/L5,202.933333,219.733333,Normal/Mild,./data/train_images/4290709089/4237840455/12.dcm,"(202.933333, 219.733333)"


# Model

Start with creating a custom dataset class

In [100]:
class SpineDataset(Dataset):
    def __init__(self, dicom_dir, coordinates, labels, transform=None):
        self.dicom_files = dicom_dir.tolist()
        self.coordinates = coordinates
        self.labels = labels
        self.transform = transform
        self.condition_mapping = {
            'Normal/Mild': 0,
            'Moderate': 1,
            'Severe': 2
        }

    def __len__(self):
        return len(self.dicom_files)

    def __getitem__(self, idx):
        dicom_path = os.path.join(self.dicom_files[idx])
        dicom = pydicom.dcmread(dicom_path)
        image = dicom.pixel_array
        image = Image.fromarray(image).convert("RGB")

        if self.transform:
            image = self.transform(image)

        coordinates = self.coordinates[idx]

        label = self.condition_mapping[self.labels[idx]]
        
        return image, coordinates, label

In [101]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 pixels
    transforms.ToTensor(),          # Convert PIL image to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet statistics

])

In [102]:
dataset = SpineDataset(train_label_coordinates_df["image_path"], train_label_coordinates_df["coordinates"], train_label_coordinates_df["severity"], transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [103]:
dataset[150]

(tensor([[[-2.0837, -2.0152, -2.0323,  ..., -2.0494, -2.0837, -2.1008],
          [-2.0494, -1.8610, -1.8782,  ..., -1.8953, -1.9809, -2.0323],
          [-2.0494, -1.9124, -1.8953,  ..., -1.9124, -1.9124, -1.9980],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],
 
         [[-2.0007, -1.9307, -1.9482,  ..., -1.9657, -2.0007, -2.0182],
          [-1.9657, -1.7731, -1.7906,  ..., -1.8081, -1.8957, -1.9482],
          [-1.9657, -1.8256, -1.8081,  ..., -1.8256, -1.8256, -1.9132],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],
 
         [[-1.7696, -1.6999, -1.7173,  ..., -1.7347, -1.7696, -1.7870],
          [-1.7347, -1.5430,

In [107]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [108]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=3):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [109]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, coordinates, labels in dataloader:
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")


KeyError: nan